# Project 4 Bean Leaves ANN

Ashton Cole

AVC697

COE 379L: Software Design for Responsive Intelligent Systems

## Description

This notebook performs a hyperparameter sweep of a densely-connected Artificial Neural Network applied to a bean leaf data set. Images of bean leaves are classified as either healthy, having rust disease, or angular leaf spot disease.

*Note: This notebook may take a long time to run. It trains and tests 16 dense neural networks.*

In [1]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import tensorflow_datasets as tfds

2024-05-02 15:47:15.729163: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 15:47:15.765753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 15:47:15.765798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 15:47:15.767122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 15:47:15.773674: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 15:47:15.774473: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Preprocessing

In [2]:
validation_split = 0.2 # Validation split
batch_size = 32
seed = 123

image_s = 500

rescale = Rescaling(scale=1.0/255)

data_train = tfds.load('beans', split='train', shuffle_files=True, as_supervised=True).batch(batch_size)
data_validation = tfds.load('beans', split='validation', shuffle_files=True, as_supervised=True).batch(batch_size)
data_test = tfds.load('beans', split='test', shuffle_files=True, as_supervised=True).batch(batch_size)

data_train_scaled = data_train.map(lambda image, label: (rescale(image), label))
data_validation_scaled = data_validation.map(lambda image, label: (rescale(image), label))
data_test_scaled = data_test.map(lambda image, label: (rescale(image), label))

## Model Construction and Compilation

In [3]:
image_length = image_s * image_s * 3

optimizer='adam'
loss='sparse_categorical_crossentropy'
metrics=['accuracy']

def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(image_s, image_s, 3))) # F1
    model.add(Dense(hp.Choice('first', [64, 128, 192, 256]), activation='relu')) # F2
    model.add(Dense(hp.Choice('second', [64, 128, 192, 256]), activation='relu')) # F3
    model.add(Dense(3, activation='softmax')) # Output
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=metrics
    )
    return model

## Model Parameter Search

In [4]:
directory = 'tuner'
project_name = 'ann-beans'

tuner = kt.GridSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    overwrite=False,
    directory=directory,
    project_name=project_name
)

tuner.search_space_summary()

Search space summary
Default search space size: 2
first (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 192, 256], 'ordered': True}
second (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 192, 256], 'ordered': True}


In [5]:
logger = CSVLogger(f'{project_name}.csv', append=True)

tuner.search(
    x=data_train_scaled,
    epochs=10,
    validation_data=data_validation_scaled,
    verbose=2,
    batch_size=batch_size,
    callbacks=[logger]
)

Trial 16 Complete [00h 07m 17s]
val_accuracy: 0.5789473652839661

Best val_accuracy So Far: 0.7819548845291138
Total elapsed time: 01h 11m 43s


## Model Testing

In [6]:
trials = tuner.oracle.trials

for (id, trial) in trials.items():
    print(f'Trial {id}')
    print(trial.summary())
    model = tuner.load_model(trial)
    test_loss, test_accuracy = model.evaluate(data_test_scaled, verbose=0)
    print(f'Test accuracy: {test_accuracy}')
    print('--------------------')

Trial 0000
Trial 0000 summary
Hyperparameters:
first: 64
second: 64
Score: 0.5714285969734192
None
Test accuracy: 0.5
--------------------
Trial 0001
Trial 0001 summary
Hyperparameters:
first: 64
second: 128
Score: 0.6240601539611816
None
Test accuracy: 0.5859375
--------------------
Trial 0002
Trial 0002 summary
Hyperparameters:
first: 64
second: 192
Score: 0.7819548845291138
None


Test accuracy: 0.7109375
--------------------
Trial 0003
Trial 0003 summary
Hyperparameters:
first: 64
second: 256
Score: 0.6766917109489441
None


Test accuracy: 0.6484375
--------------------
Trial 0004
Trial 0004 summary
Hyperparameters:
first: 128
second: 64
Score: 0.6766917109489441
None


Test accuracy: 0.671875
--------------------
Trial 0005
Trial 0005 summary
Hyperparameters:
first: 128
second: 128
Score: 0.6090225577354431
None


Test accuracy: 0.609375
--------------------
Trial 0006
Trial 0006 summary
Hyperparameters:
first: 128
second: 192
Score: 0.5639097690582275
None


...

Test accuracy: 0.546875
--------------------
Trial 0012
Trial 0012 summary
Hyperparameters:
first: 256
second: 64
Score: 0.61654132604599
None


Test accuracy: 0.578125
--------------------
Trial 0013
Trial 0013 summary
Hyperparameters:
first: 256
second: 128
Score: 0.7067669034004211
None


Test accuracy: 0.7109375
--------------------
Trial 0014
Trial 0014 summary
Hyperparameters:
first: 256
second: 192
Score: 0.7518796920776367
None


Test accuracy: 0.6953125
--------------------
Trial 0015
Trial 0015 summary
Hyperparameters:
first: 256
second: 256
Score: 0.5789473652839661
None


Test accuracy: 0.515625
--------------------
